# Notebook to show off as many use cases / edge cases of uncertainty as we can think of (will form basis of pytest module later)

In [1]:
import os
import sys
import pickle
import copy
import numpy as np
import rmgpy.rmg.input  # note this method is currently broken for multiple reactors in an input file
import rmgpy.rmg.main
import rmgpy.chemkin
import rmgpy.tools.uncertainty
import rmgpy.kinetics.uncertainties

import rmgpy.tools.canteramodel
import random

import rmgpy.kinetics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_i_thing(thing, thing_list):
    for i in range(len(thing_list)):
        if thing.is_isomorphic(thing_list[i]):
            return i
    raise AssertionError(f'Could not find {thing} in list of {len(thing_list)} things')

In [47]:
def pad_to_length(input_str, length):
    input_str = str(input_str)
    if len(input_str) > length:
        print('Pad length for this string must be at least', len(input_str))
        raise ValueError
    return input_str + ' ' * (length - len(input_str))

In [4]:
CH4 = rmgpy.species.Species(smiles='C')
CH3 = rmgpy.species.Species(smiles='[CH3]')
C2H4 = rmgpy.species.Species().from_smiles('C=C')

## Load Mech_00_ethane

In [5]:
# Must use annotated chemkin file
chemkin_file = './mech_00_ethane/chemkin/chem_annotated.inp'
mech_dir = os.path.dirname(chemkin_file)
dict_file = os.path.join(mech_dir, 'species_dictionary.txt')
input_file = os.path.join(os.path.dirname(mech_dir), 'input.py')

# Run Gao estimation of input parameters (takes a long time to load database)
uncertainty = rmgpy.tools.uncertainty.Uncertainty(
    output_directory=os.path.join(mech_dir, 'uncertainty')
)
uncertainty.load_model(chemkin_file, dict_file)

In [6]:
# load the database either copying libraries manually or using the input file

read_from_input = True

if read_from_input:

    global rmg0
    rmg0 = rmgpy.rmg.main.RMG()
    rmg0.output_directory = '.'
    
    rmgpy.rmg.input.read_input_file(input_file, rmg0)

    thermo_libs = rmg0.thermo_libraries
    kinetics_libs = [x[0] if isinstance(x, tuple) else x for x in rmg0.reaction_libraries]
    
    
    kinetics_families = rmg0.kinetics_families

else:
    # --------------- CAUTION!!! Databases here must match the ones used to generate the mechanism
    thermo_libs = [
        'primaryThermoLibrary',
    ]
    kinetics_libs = [
        'BurkeH2O2inN2',
    ]
    
    kinetics_families = [
        'default',
    ]
assert set(thermo_libs) == set(['primaryThermoLibrary'])
assert set(kinetics_libs) == set(['BurkeH2O2inN2'])
assert kinetics_families == 'default'
print('Thermo Libraries:', thermo_libs)
print('Kinetics Libraries:', kinetics_libs)
print('Kinetics Families:', kinetics_families)


uncertainty.load_database(  # this call does the averaging up of kinetics families
    thermo_libraries=thermo_libs,
    kinetics_families=kinetics_families,
    reaction_libraries=kinetics_libs,
)

Thermo Libraries: ['primaryThermoLibrary']
Kinetics Libraries: ['BurkeH2O2inN2']
Kinetics Families: default


ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([60.2599,68.0494,74.7775,80.9311,90.8846,97.4337,105.393],'J/(mol*K)'), H298=(-477.191,'kJ/mol'), S298=(269.551,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(103.931,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-(Cds-Cd)(Cds-Cd)) + group(O2s-(Cds-O2d)H) + group(Cds-OdOsOs) + group(Li-OCOdO) + radical(OC=OOJ)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([60.2599,68.0494,74.7775,80.9311,90.8846,97.4337,105.393],'J/(mol*K)'), H298=(-477.191,'kJ/mol'), S298=(269.551,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(103.931,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-(Cds-Cd)(Cds-Cd)) + group(O2s-(Cds-O2d)H) + group(Cds-OdOsOs) + group(Li-OCOdO) + radical(OC=OOJ)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for En

In [7]:
# make sure CH3 hasn't been added to primaryThermoLibrary
thermo_lib_item_list = [uncertainty.database.thermo.libraries['primaryThermoLibrary'].entries[key].item \
                            for key in uncertainty.database.thermo.libraries['primaryThermoLibrary'].entries.keys()]
assert not any([CH3.is_isomorphic(item) for item in thermo_lib_item_list]), "You need a new test if CH3 gets into the primaryThermoLibrary"

### Uncorrelated test cases

In [21]:
# Get the different kinetic and thermo sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()
uncertainty.compile_all_sources()

In [22]:
uncertainty.all_thermo_sources

{'Library': [0, 1, 2, 4, 5, 6, 7, 8, 9, 10],
 'QM': [],
 'GAV': {'group': [<Entry index=309 label="Cds-CdsHH">,
   <Entry index=921 label="Cs-CsHHH">],
  'radical': [<Entry index=840 label="CCJ">, <Entry index=796 label="CH3">]},
 'ADS': {}}

# See at a glance what kind of species estimates are in the mech

In [54]:
print('Index', '\tSpecies', '\tSources', '\tUncertainty kcal/mol')
print('----------------------------------------------------------')
for i in range(len(uncertainty.species_list)):
    print(i, '\t',
          pad_to_length(uncertainty.species_list[i], 14),
          pad_to_length(set(uncertainty.species_sources_dict[uncertainty.species_list[i]].keys()), 18),
          '\t', f'{uncertainty.thermo_input_uncertainties[i]:.4f}'
         )

Index 	Species 	Sources 	Uncertainty kcal/mol
----------------------------------------------------------
0 	 Ar             {'Library'}        	 1.5000
1 	 Ne             {'Library'}        	 1.5000
2 	 N2             {'Library'}        	 1.5000
3 	 ethane(1)      {'GAV'}            	 1.5133
4 	 O2(2)          {'Library'}        	 1.5000
5 	 H(3)           {'Library'}        	 1.5000
6 	 O(4)           {'Library'}        	 1.5000
7 	 OH(5)          {'Library'}        	 1.5000
8 	 H2(6)          {'Library'}        	 1.5000
9 	 H2O(7)         {'Library'}        	 1.5000
10 	 C(12)          {'Library'}        	 1.5000
11 	 [CH3](13)      {'Library', 'GAV'} 	 2.1237
12 	 C[CH2](14)     {'GAV'}            	 1.5166
13 	 C=C(21)        {'GAV'}            	 1.5133


In [67]:
uncertainty.species_sources_dict[uncertainty.species_list[3]]

{'GAV': {'group': [(<Entry index=921 label="Cs-CsHHH">, 2)]}}

In [68]:
np.sqrt(1.5 ** 2.0 + 2.0 ** 2.0 * 0.1 ** 2.0)

1.5132745950421556

# Examples of library-only species

In [57]:
i_CH4 = get_i_thing(CH4, uncertainty.species_list)

library_only_species_indices = []
for i in range(len(uncertainty.species_list)):
# for i in [i_CH4]:

    source_entry = uncertainty.species_sources_dict[uncertainty.species_list[i]]

    # should have 'Library' as only key
    if set(['Library']) != set(source_entry.keys()):
        continue
    
    # print(i, uncertainty.species_list[i])
    display(uncertainty.species_list[i])
    
    print('Source Dict:', source_entry)
    print('Uncertainty:', uncertainty.thermo_input_uncertainties[i], 'kcal/mol')
    
    # the number is the index of the species in the mechanism
    # but this also isn't really used anywhere to my knowledge
    assert i == source_entry['Library']
    library_only_species_indices.append(i)
    print()
    print()

assert i_CH4 in library_only_species_indices

Source Dict: {'Library': 0}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 1}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 2}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 4}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 5}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 6}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 7}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 8}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 9}
Uncertainty: 1.5 kcal/mol




Source Dict: {'Library': 10}
Uncertainty: 1.5 kcal/mol




# Examples of Group Additivity-only species

In [58]:
i_C2H4 = get_i_thing(C2H4, uncertainty.species_list)

GAV_only_species_indices = []
for i in range(len(uncertainty.species_list)):
    source_entry = uncertainty.species_sources_dict[uncertainty.species_list[i]]

    # should have 'GAV' as only key
    if set(['GAV']) != set(source_entry.keys()):
        continue
    
    # print(i, uncertainty.species_list[i])
    display(uncertainty.species_list[i])
    
    print('Source Dict:', source_entry)
    print('Uncertainty:', uncertainty.thermo_input_uncertainties[i], 'kcal/mol')
    
    GAV_only_species_indices.append(i)
    print()
    print()

assert i_C2H4 in GAV_only_species_indices

Source Dict: {'GAV': {'group': [(<Entry index=921 label="Cs-CsHHH">, 2)]}}
Uncertainty: 1.5132745950421556 kcal/mol




Source Dict: {'GAV': {'group': [(<Entry index=921 label="Cs-CsHHH">, 2)], 'radical': [(<Entry index=840 label="CCJ">, 1)]}}
Uncertainty: 1.51657508881031 kcal/mol




Source Dict: {'GAV': {'group': [(<Entry index=309 label="Cds-CdsHH">, 2)]}}
Uncertainty: 1.5132745950421556 kcal/mol




In [26]:
i_CH4

10

In [ ]:
CH4_source_entry = uncertainty.species_sources_dict[uncertainty.species_list[i_CH4]]
CH3_source_entry = uncertainty.species_sources_dict[uncertainty.species_list[i_CH3]]
C2H4_source_entry = uncertainty.species_sources_dict[uncertainty.species_list[i_C2H4]]


In [ ]:
uncertainty.database.thermo

In [ ]:
# uncertainty.species_sources_dict

In [ ]:
rmgpy.tools.uncertainty.ThermoParameterUncertainty().dG_library

In [ ]:
uncertainty.species_sources_dict[uncertainty.species_list[i_C2H4]]

In [ ]:
# spot check som

In [ ]:
# maybe want to reformat this as a pytest thing to better resuse code

In [ ]:
uncertainty.thermo_input_uncertainties

In [ ]:
uncertainty.species_list